## Create Responsible AI Dashboard

In [ ]:
# Import credentials
from resource_credentials import (subscription_key, 
    resource_group_name, 
    workspace_name)

from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

# Create a ml client
ml_client = MLClient(credential=DefaultAzureCredential(), 
                     subscription_id=subscription_key, 
                     resource_group_name=resource_group_name,
                     workspace_name=workspace_name)

### Separate train and test data

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
import os

# Split data into training and testing sets
diabetes_data = pd.read_csv('./src/ml-table/diabetes.csv')

diabetes_data.drop('PatientID', axis=1, inplace=True)

train_data, test_data = train_test_split(diabetes_data, test_size=0.2, random_state=42, shuffle=True)

# Save train and test data to csv
if not os.path.exists('./src/ml-table-train'):
    os.makedirs('./src/ml-table-train')
if not os.path.exists('./src/ml-table-test'):
    os.makedirs('./src/ml-table-test')
    
train_data.to_csv('./src/ml-table-train/diabetes-train.csv', index=False)
test_data.to_csv('./src/ml-table-test/diabetes-test.csv', index=False)

In [ ]:
from azure.ai.ml.entities import Data
from azure.ai.ml.constants import AssetTypes

# Register train dataset
data_asset_path = "./src/ml-table-train"

data = Data(path=data_asset_path, 
            type=AssetTypes.MLTABLE,
            name="diabetes-table-train")

train_data = ml_client.data.create_or_update(data)

# Register test dataset
data_asset_path = "./src/ml-table-test"

data = Data(path=data_asset_path, 
            type=AssetTypes.MLTABLE,
            name="diabetes-table-test")

test_data = ml_client.data.create_or_update(data)

### Get model

In [ ]:
model = ml_client.models.get("mlflow-diabetes-model", version=1)

model_name = model.name
expected_model_id = f"{model_name}:1"
azureml_model_id = f"azureml:{expected_model_id}"

print(model.name)

### Import dashboard components


In [ ]:
## Create registry client
ml_registry_client = MLClient(credential=DefaultAzureCredential(), 
                     subscription_id=subscription_key, 
                     resource_group_name=resource_group_name,
                     registry_name="azureml")

In [ ]:
rai_constructor_component = ml_registry_client.components.get(
    name="microsoft_azureml_rai_tabular_insight_constructor", label="latest"
)

print(rai_constructor_component.version)

rai_erroranalysis_component = ml_registry_client.components.get(
    name="microsoft_azureml_rai_tabular_erroranalysis", version=rai_constructor_component.version
)

rai_explanation_component = ml_registry_client.components.get(
    name="microsoft_azureml_rai_tabular_explanation", version=rai_constructor_component.version
)

rai_gather_component = ml_registry_client.components.get(
    name="microsoft_azureml_rai_tabular_insight_gather", version=rai_constructor_component.version
)

### Create pipeline

In [ ]:
from azure.ai.ml.dsl import pipeline
from azure.ai.ml.constants import AssetTypes
from azure.ai.ml import Input, Output

@pipeline(experiment_name="rai-diabetes-pipeline", compute="compute-instance-sdk")
def rai_decision_pipeline(target_column_name, train_data, test_data):
     # Initiate the RAIInsights
    create_rai_job = rai_constructor_component(
        title="RAI dashboard diabetes",
        task_type="classification",
        model_info=expected_model_id,
        model_input=Input(type=AssetTypes.MLFLOW_MODEL, path=azureml_model_id),
        train_dataset=train_data,
        test_dataset=test_data,
        target_column_name=target_column_name
    )
    create_rai_job.set_limits(timeout=300)

    # Add error analysis
    error_job = rai_erroranalysis_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
    )
    error_job.set_limits(timeout=300)

    # Add explanations
    explanation_job = rai_explanation_component(
        rai_insights_dashboard=create_rai_job.outputs.rai_insights_dashboard,
        comment="add explanation", 
    )
    explanation_job.set_limits(timeout=300)

    # Combine everything
    rai_gather_job = rai_gather_component(
        constructor=create_rai_job.outputs.rai_insights_dashboard,
        insight_3=error_job.outputs.error_analysis,
        insight_4=explanation_job.outputs.explanation,
    )
    rai_gather_job.set_limits(timeout=300)

    rai_gather_job.outputs.dashboard.mode = "upload"

    return {
        "dashboard": rai_gather_job.outputs.dashboard,
    }

### Create inputs and submit pipeline

In [ ]:
import uuid
target_feature = "Diabetic"

diabetes_train = Input(
    type=AssetTypes.MLTABLE,
    path=f"azureml:diabetes-table-train:{train_data.version}",
    mode="download",
)
diabetes_test = Input(
    type=AssetTypes.MLTABLE,
    path=f"azureml:diabetes-table-test:{test_data.version}",
    mode="download",
)

rai_pipeline = rai_decision_pipeline(
    target_column_name=target_feature,
    train_data=diabetes_train,
    test_data=diabetes_test
)

rand_path = str(uuid.uuid4())
rai_pipeline.outputs.dashboard = Output(
    path=f"azureml://datastores/workspaceblobstore/paths/{rand_path}/dashboard/",
    mode="upload",
    type="uri_folder",
)

rai_pipeline_job = ml_client.jobs.create_or_update(rai_pipeline)